In [ ]:
# (Query Expansion, Self-Query Retrieval) -> (Hybrid-search) -> Chunks -> Re-ranking(#ColBERTv2) -> (Prompt+Chunks) -> LLM(gpt-4o)

In [1]:
!pip install -qU langchain langchain_community==0.2.6 fastembed==0.3.2 langchain_core langchain_milvus langchain_openai flashrank lark faiss-cpu

In [91]:
# Query Expansion modules
import os
import time
from typing import List, Dict, Literal, Any, Optional, Tuple
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Milvus
from langchain_openai import ChatOpenAI

# SparseEmbeddiing Class modules
from pydantic import BaseModel, Field, model_validator, validator
from operator import itemgetter
from langchain_core.globals import set_debug
from langchain_core.embeddings import Embeddings

# Custom Query Expansion Class modules
from abc import ABC, abstractmethod
from pydantic import BaseModel
from typing import List
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts import BasePromptTemplate
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_core.language_models import BaseLanguageModel
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
import re

from langchain_core.documents import Document
from pymilvus import Collection, utility, AnnSearchRequest, RRFRanker, connections

# Reranker modules
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors.flashrank_rerank import FlashrankRerank
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS

# LECL chain modules
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from IPython.display import display, Markdown
from langchain.callbacks import get_openai_callback
from langchain_core.messages import AIMessage

In [92]:
os.environ['OPENAI_API_KEY'] = "111e976694f24802a152f35e531bc6c2"

In [93]:
## Define the Credentials
OPENAI_API_KEY = ""
OPENAI_API_BASE = ""
ZILLIZ_CLOUD_URI = ""          # os.environ['ZILLIZ_URI'] #"https://in03-c2cc7c5da8decab.api.gcp-us-west1.zillizcloud.com"
ZILLIZ_CLOUD_API_KEY = "9"    
COLLECTION_NAME=""

In [94]:
DENSE_EMBEDDING_MODEL = "jinaai/jina-embeddings-v2-base-en"
SPARSE_EMBEDDING_MODEL = "Qdrant/bm42-all-minilm-l6-v2-attentions"
LLM_MODEL = "gpt-4o"
TEMPERATURE = 0.0
langchain=True
DENSE_SEARCH_PARAMS = {
    "index_type": "IVF_SQ8",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

SPARSE_SEARCH_PARAMS = {
        "index_type": "SPARSE_INVERTED_INDEX",
        "metric_type": "IP",
    }

# Defone your Question Here
QUESTION = "tell me about Advanced manufacturing?"

In [95]:
class SparseFastEmbedEmbeddings(BaseModel, Embeddings):
    """Qdrant FastEmbedding models.
    FastEmbed is a lightweight, fast, Python library built for embedding generation.

    To use this class, you must install the `fastembed` Python package.

    `pip install fastembed`
    Example:
        from langchain_community.embeddings import FastEmbedEmbeddings
        fastembed = FastEmbedEmbeddings()
    """

    model_name: str = "BAAI/bge-small-en-v1.5"
    """Name of the FastEmbedding model to use
    Defaults to "BAAI/bge-small-en-v1.5"
    Find the list of supported models at
    https://qdrant.github.io/fastembed/examples/Supported_Models/
    """

    cache_dir: Optional[str] = Field(default=None)
    """The path to the cache directory.
    Defaults to `local_cache` in the parent directory
    """

    threads: Optional[int] = Field(default=None)
    """The number of threads single onnxruntime session can use.
    Defaults to None
    """

    doc_embed_type: str = "default"
    """Type of embedding to use for documents
    The available options are: "default" and "passage"
    """

    model: Any = Field(default=None, exclude=True)  # Renamed to 'model' and marked as private

    class Config:
        """Configuration for this pydantic object."""
        extra = 'forbid'

    @model_validator(mode='before')
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that FastEmbed has been installed."""
        return values

    def __init__(self, **data):
        super().__init__(**data)
        self._initialize_model()

    def _initialize_model(self):
        """Initialize the FastEmbed model."""
        try:
            # >= v0.2.0
            from fastembed import SparseTextEmbedding

            self.model = SparseTextEmbedding(
                model_name=self.model_name,
                cache_dir=self.cache_dir,
                threads=self.threads,
            )
        except ImportError as ie:
            try:
                # < v0.2.0
                from fastembed.embedding import FlagEmbedding

                self.model = FlagEmbedding(
                    model_name=self.model_name,
                    cache_dir=self.cache_dir,
                    threads=self.threads,
                )
            except ImportError:
                raise ImportError(
                    "Could not import 'fastembed' Python package. "
                    "Please install it with `pip install fastembed`."
                ) from ie

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for documents using FastEmbed.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """
        embeddings: List[np.ndarray]
        if self.doc_embed_type == "passage":
            embeddings = self.model.passage_embed(texts)
        else:
            embeddings = self.model.embed(texts)
        return [
            {int(idx): float(val) for idx, val in zip(embed.indices, embed.values)}
            for embed in embeddings
        ]

    def embed_query(self, text: str) -> List[float]:
        """Generate query embeddings using FastEmbed.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        query_embeddings: np.ndarray = next(self.model.query_embed(text))
        return [{int(idx): float(val) for idx, val in zip(query_embeddings.indices, query_embeddings.values)}]

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [96]:
# Custom Query Expansion Class
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        cleaned_lines = [re.sub(r'^\d+\.\s*', '', line) for line in lines]
        return cleaned_lines

class CustomMultiQueryRetriever(MultiQueryRetriever):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    MULTI_QUERY_PROMPT = PromptTemplate(
        input_variables = ["question"],
        template="""You are an AI language model assistant. Your task is to break down question if possible to retrieve
       relevant documents from a vector database. By generating multiple break down questions of user question, your goal is to 
       help the user overcome some of the limitations of distance-based similarity search. Provide these multi step questions
       separated by newlines. Only break down question if poosible otherwise return the original question.\n Original Question: {question}
        """ 
    )

    @classmethod
    def from_llm(
        cls,
        retriever: BaseRetriever,
        llm: Any,
        prompt: BasePromptTemplate = MULTI_QUERY_PROMPT,
        include_original: bool = True,
    ) -> "CustomMultiQueryRetriever":

        output_parser = LineListOutputParser()
        llm_chain = prompt | llm | output_parser

        return cls(
            retriever=retriever,
            llm_chain=llm_chain,
            include_original=include_original
        )

In [97]:
def sparse_embedding_model(texts: List[str], embed_model):
    embeddings = SparseFastEmbedEmbeddings(model_name=embed_model)
    query_embeddings = embeddings.embed_documents([texts])
    return query_embeddings

def dense_embedding_model(texts: List[str], embed_model):
    embeddings = FastEmbedEmbeddings(model_name=embed_model)
    query_embeddings = embeddings.embed_documents([texts])
    return query_embeddings

In [98]:
def retrieval_embedding_model(model_name):
    embed_model = FastEmbedEmbeddings(model_name=model_name)
    return embed_model

def initialise_vector_store(vector_field:str, search_params:dict):
    embeddings = retrieval_embedding_model(DENSE_EMBEDDING_MODEL)
    vector_store = Milvus(
           embeddings,
           connection_args={"uri": ZILLIZ_CLOUD_URI, 'token': ZILLIZ_CLOUD_API_KEY, 'secure': True},
           collection_name = COLLECTION_NAME, ## custom collection name 
           search_params = search_params,
            vector_field = vector_field
        )
    return vector_store

def initialise_llm_model(llm_model):
    llm_model = ChatOpenAI(model=llm_model,openai_api_key=OPENAI_API_KEY, temperature=TEMPERATURE, openai_api_base=OPENAI_API_BASE)
    return llm_model

In [99]:
# Self Query Retriever
def Self_query_retrieval(question):
    llm_model = initialise_llm_model(LLM_MODEL)
    vector_store = initialise_vector_store("dense_vector", DENSE_SEARCH_PARAMS)
    metadata_field_info = [
        AttributeInfo(
            name="source_link",
            description="Defines the source link of the file.",
            type="string",
        ),
        AttributeInfo(
            name="author_name",
            description="the local file path of the file.",
            type="string",
        ),
        AttributeInfo(
            name="related_topics",
            description="Total number of files pages present inside the file.",
            type="array",
        ),
        AttributeInfo(
            name="pdf_links", 
            description="The year the file was released or published.", 
            type="array"
        ),
    ]
    document_content_description = "Brief summary of a file."
    selfq_retriever = SelfQueryRetriever.from_llm(
        llm_model, vector_store, document_content_description, metadata_field_info, verbose=True
    )
    structured_query = selfq_retriever.query_constructor.invoke({"query": question})
    new_query, search_kwargs = selfq_retriever._prepare_query(query, structured_query)
    return new_query, search_kwargs

In [100]:
def Custom_Query_Exapander(question) -> List:
    llm_model = initialise_llm_model(LLM_MODEL)
    vector_store = initialise_vector_store("dense_vector", DENSE_SEARCH_PARAMS)
    retriever_obj = CustomMultiQueryRetriever.from_llm(
        retriever=vector_store.as_retriever(),
        llm = llm_model,
        include_original = True
    )
    run_manager = CallbackManagerForRetrieverRun(run_id="example_run", handlers=[], inheritable_handlers={})
    multiq_queries = retriever_obj.generate_queries(question, run_manager)
    return multiq_queries

In [101]:
def load_collection(collection_name):
    connections.connect(
        uri=ZILLIZ_CLOUD_URI,
        token=ZILLIZ_CLOUD_API_KEY
    )

    milvus_collection = Collection(name=collection_name)
    milvus_collection.load()
    return milvus_collection

In [102]:
# Hybrid Search
def milvus_hybrid_search(question, expr):
    milvus_collection = load_collection(COLLECTION_NAME)
    sparse_question_emb = sparse_embedding_model(question, SPARSE_EMBEDDING_MODEL)
    dense_question_emb = dense_embedding_model(question, DENSE_EMBEDDING_MODEL)
    output = []
    
    sparse_q = AnnSearchRequest(sparse_question_emb, "sparse_vector", SPARSE_SEARCH_PARAMS, limit=3) #expr
    dense_q = AnnSearchRequest(dense_question_emb, "dense_vector", DENSE_SEARCH_PARAMS, limit=3) #expr

    res = milvus_collection.hybrid_search([sparse_q, dense_q], rerank=RRFRanker(), limit=6,
            output_fields=["source_link", "text", "author_name", "related_topics", "pdf_links"]  # Include title field in result
        )
    print(f"Hybrid Search Result: {res}")
    for _, hits in enumerate(res):
        for hit in hits:
            page_content = hit.entity.get("text")
            metadata = {
                "source_link": hit.entity.get("source_link"),
                "author_name": hit.entity.get("author_name"),
                "related_topics": hit.entity.get("related_topics"),
                "pdf_links": hit.entity.get("pdf_links")
                }
            doc_chunk = Document(page_content=page_content, metadata=metadata)
            output.append(doc_chunk)
    return output

In [103]:
# Reranker
def faiss_store_docs_to_rerank(docs_to_rerank):
    embeddings = retrieval_embedding_model(model_name=DENSE_EMBEDDING_MODEL)
    retriever = FAISS.from_documents(docs_to_rerank, embeddings)
    return retriever

def Reranker(question, docs_to_rerank) -> List:
    FlashrankRerank.update_forward_refs()
    compressor = FlashrankRerank()
    retriever = faiss_store_docs_to_rerank(docs_to_rerank)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever.as_retriever()
    )
    
    compressed_docs = compression_retriever.invoke(
        question
    )
    return compressed_docs

In [104]:
def format_document(doc: Document) -> str:
        # if 'prechunk' in doc.metadata.keys():
        #     prompt = PromptTemplate(input_variables=["prechunk"], template="{prechunk}\n")
        prompt = PromptTemplate(input_variables=["page_content"], template="{page_content}")
        # if 'postchunk' in doc.metadata.keys():
        #     prompt += PromptTemplate(input_variables=["postchunk"], template="{postchunk}\n")
        if 'source_link' in doc.metadata.keys():
            prompt += PromptTemplate(input_variables=["source_link"], template="\n[Source: {source_link}]")
        base_info = {"page_content": doc.page_content, **doc.metadata}
        missing_metadata = set(prompt.input_variables).difference(base_info)
        if len(missing_metadata) > 0:
            required_metadata = [
                iv for iv in prompt.input_variables if iv != "page_content"
            ]
            raise ValueError(
                f"Document prompt requires documents to have metadata variables: "
                f"{required_metadata}. Received document with missing metadata: "
                f"{list(missing_metadata)}."
            )
        
        return prompt.format(**base_info)

def format_docs(docs):
    return "\n\n".join(format_document(doc) for doc in docs)

In [105]:
combined_docs = [
    Document(metadata={'source_link': 'https://docs.nvidia.com/cuda/cuda-toolkit-release-notes/index.html', 'prechunk': ['', ''], 'postchunk': ['* cublasLtMatmul ignored the CUBLASLT_MATMUL_DESC_AMAX_D_POINTER for unsupported configurations instead of returning an error. In particular, computing absolute maximum of D is currently supported only for FP8 Matmul when the output data type is also FP8 (CUDA_R_8F_E4M3 or CUDA_R_8F_E5M2). cublasLtMatmul ignored the CUBLASLT_MATMUL_DESC_AMAX_D_POINTER for unsupported configurations instead of returning an error. In particular, computing absolute maximum of D is currently supported only for FP8 Matmul when the output data type is also FP8 (CUDA_R_8F_E4M3 or CUDA_R_8F_E5M2). * Reduced host-side overheads for some of the cuBLASLt APIs: cublasLtMatmul(), cublasLtMatmulAlgoCheck(), and cublasLtMatmulAlgoGetHeuristic(). The issue was introduced in CUDA Toolkit 12.4. Reduced host-side overheads for some of the cuBLASLt APIs: cublasLtMatmul(), cublasLtMatmulAlgoCheck(), and cublasLtMatmulAlgoGetHeuristic(). The issue was introduced in CUDA Toolkit 12.4. * cublasLtMatmul() and cublasLtMatmulAlgoGetHeuristic() could have resulted in floating point exceptions (FPE) on some Hopper-based GPUs, including Multi-Instance GPU (MIG). The issue was introduced in cuBLAS 11.8. cublasLtMatmul() and cublasLtMatmulAlgoGetHeuristic() could have resulted in floating point exceptions (FPE) on some Hopper-based GPUs, including Multi-Instance GPU (MIG). The issue was introduced in cuBLAS 11.8. 2.1.4. CUBLAS: RELEASE 12.4\uf0c1 * New Features cuBLAS adds experimental APIs to support grouped batched GEMM for single precision and double precision. Single precision also supports the math mode, CUBLAS_TF32_TENSOR_OP_MATH. Grouped batch mode allows you to concurrently solve GEMMs of different dimensions (m, n, k), leading dimensions (lda, ldb, ldc), transpositions (transa, transb), and scaling factors (alpha, beta). Please see gemmGroupedBatched for more details. New Features * cuBLAS adds experimental APIs to support grouped batched GEMM for single precision and double precision. Single precision also supports the math mode, CUBLAS_TF32_TENSOR_OP_MATH. Grouped batch mode allows you to concurrently solve GEMMs of different dimensions (m, n, k), leading dimensions (lda, ldb, ldc), transpositions (transa, transb), and scaling factors (alpha, beta). Please see gemmGroupedBatched for more details. cuBLAS adds experimental APIs to support grouped batched GEMM for single precision and double precision. Single precision also supports the math mode, CUBLAS_TF32_TENSOR_OP_MATH. Grouped batch mode allows you to concurrently solve GEMMs of different dimensions (m, n, k), leading dimensions (lda, ldb, ldc), transpositions (transa, transb), and scaling factors (alpha, beta). Please see gemmGroupedBatched for more details. * Known Issues When the current context has been created using cuGreenCtxCreate(), cuBLAS does not properly detect the number of SMs available. The user may provide the corrected SM count to cuBLAS using an API such as cublasSetSmCountTarget(). BLAS level 2 and 3 functions might not treat alpha in a BLAS compliant manner when alpha is zero and the pointer mode is set to CUBLAS_POINTER_MODE_DEVICE. This is the same known issue documented in cuBLAS 12.3 Update 1. cublasLtMatmul with K equals 1 and epilogue CUBLASLT_EPILOGUE_D{RELU,GELU}_BGRAD could out-of-bound access the workspace. The issue exists since cuBLAS 11.3 Update 1. cublasLtMatmul with K equals 1 and epilogue CUBLASLT_EPILOGUE_D{RELU,GELU} could produce illegal memory access if no workspace is provided. The issue exists since cuBLAS 11.6. When captured in CUDA Graph stream capture, cuBLAS routines can create memory nodes through the use of stream-ordered allocation APIs, cudaMallocAsync and cudaFreeAsync. However, as there is currently no support for memory nodes in child graphs or graphs launched from the device, attempts to capture cuBLAS routines in such scenarios may fail. To avoid this issue, use the cublasSetWorkspace() function to provide user-owned workspace memory. Known Issues * When the current context has been created using cuGreenCtxCreate(), cuBLAS does not properly detect the number of SMs available. The user may provide the corrected SM count to cuBLAS using an API such as cublasSetSmCountTarget(). When the current context has been created using cuGreenCtxCreate(), cuBLAS does not properly detect the number of SMs available. The user may provide the corrected SM count to cuBLAS using an API such as cublasSetSmCountTarget(). * BLAS level 2 and 3 functions might not treat alpha in a BLAS compliant manner when alpha is zero and the pointer mode is set to CUBLAS_POINTER_MODE_DEVICE. This is the same known issue documented in cuBLAS 12.3 Update 1. BLAS level 2 and 3 functions might not treat alpha in a BLAS compliant manner when alpha is zero and the pointer mode is set to CUBLAS_POINTER_MODE_DEVICE. This is the same known issue documented in cuBLAS 12.3 Update 1. * cublasLtMatmul with K equals 1 and epilogue CUBLASLT_EPILOGUE_D{RELU,GELU}_BGRAD could out-of-bound access the workspace. The issue exists since cuBLAS 11.3 Update 1. cublasLtMatmul with K equals 1 and epilogue CUBLASLT_EPILOGUE_D{RELU,GELU}_BGRAD could out-of-bound access the workspace. The issue exists since cuBLAS 11.3 Update 1. * cublasLtMatmul with K equals 1 and epilogue CUBLASLT_EPILOGUE_D{RELU,GELU} could produce illegal memory access if no workspace is provided. The issue exists since cuBLAS 11.6. cublasLtMatmul with K equals 1 and epilogue CUBLASLT_EPILOGUE_D{RELU,GELU} could produce illegal memory access if no workspace is provided. The issue exists since cuBLAS 11.6. * When captured in CUDA Graph stream capture, cuBLAS routines can create memory nodes through the use of stream-ordered allocation APIs, cudaMallocAsync and cudaFreeAsync. However, as there is currently no support for memory nodes in child graphs or graphs launched from the device, attempts to capture cuBLAS routines in such scenarios may fail. To avoid this issue, use the cublasSetWorkspace() function to provide user-owned workspace memory. When captured in CUDA Graph stream capture, cuBLAS routines can create memory nodes through the use of stream-ordered allocation APIs, cudaMallocAsync and cudaFreeAsync. However, as there is currently no support for memory nodes in child graphs or graphs launched from the device, attempts to capture cuBLAS routines in such scenarios may fail. To avoid this issue, use the cublasSetWorkspace() function to provide user-owned workspace memory. 2.1.5. CUBLAS: RELEASE 12.3 UPDATE 1\uf0c1 * New Features Improved performance of heuristics cache for workloads that have a high eviction rate. New Features * Improved performance of heuristics cache for workloads that have a high eviction rate. Improved performance of heuristics cache for workloads that have a high eviction rate. * Known Issues BLAS level 2 and 3 functions might not treat alpha in a BLAS compliant manner when alpha is zero and the pointer mode is set to CUBLAS_POINTER_MODE_DEVICE. The expected behavior is that the corresponding computations would be skipped. You may encounter the following issues: (1) HER{,2,X,K,2K} may zero the imaginary part on the diagonal elements of the output matrix; and (2) HER{,2,X,K,2K}, SYR{,2,X,K,2K} and others may produce NaN resulting from performing computation on matrices A and B which would otherwise be skipped. If strict compliance with BLAS is required, the user may manually check for alpha value before invoking the functions or switch to CUBLAS_POINTER_MODE_HOST. Known Issues * BLAS level 2 and 3 functions might not treat alpha in a BLAS compliant manner when alpha is zero and the pointer mode is set to CUBLAS_POINTER_MODE_DEVICE. The expected behavior is that the corresponding computations would be skipped. You may encounter the following issues: (1) HER{,2,X,K,2K} may zero the imaginary part on the diagonal elements of the output matrix; and (2) HER{,2,X,K,2K}, SYR{,2,X,K,2K} and others may produce NaN resulting from performing computation on matrices A and B which would otherwise be skipped.', 'If strict compliance with BLAS is required, the user may manually check for alpha value before invoking the functions or switch to CUBLAS_POINTER_MODE_HOST. BLAS level 2 and 3 functions might not treat alpha in a BLAS compliant manner when alpha is zero and the pointer mode is set to CUBLAS_POINTER_MODE_DEVICE. The expected behavior is that the corresponding computations would be skipped. You may encounter the following issues: (1) HER{,2,X,K,2K} may zero the imaginary part on the diagonal elements of the output matrix; and (2) HER{,2,X,K,2K}, SYR{,2,X,K,2K} and others may produce NaN resulting from performing computation on matrices A and B which would otherwise be skipped. If strict compliance with BLAS is required, the user may manually check for alpha value before invoking the functions or switch to CUBLAS_POINTER_MODE_HOST. * Resolved Issues cuBLASLt matmul operations might have computed the output incorrectly under the following conditions: the data type of matrices A and B is FP8, the data type of matrices C and D is FP32, FP16, or BF16, the beta value is 1.0, the C and D matrices are the same, the epilogue contains GELU activation function. When an application compiled with cuBLASLt from CUDA Toolkit 12.2 update 1 or earlier runs with cuBLASLt from CUDA Toolkit 12.2 update 2 or CUDA Toolkit 12.3, matrix multiply descriptors initialized using cublasLtMatmulDescInit() sometimes did not respect attribute changes using cublasLtMatmulDescSetAttribute(). Fixed creation of cuBLAS or cuBLASLt handles on Hopper GPUs under the Multi-Process Service (MPS). cublasLtMatmul with K equals 1 and epilogue CUBLASLT_EPILOGUE_BGRAD{A,B} might have returned incorrect results for the bias gradient. Resolved Issues * cuBLASLt matmul operations might have computed the output incorrectly under the following conditions: the data type of matrices A and B is FP8, the data type of matrices C and D is FP32, FP16, or BF16, the beta value is 1.0, the C and D matrices are the same, the epilogue contains GELU activation function. cuBLASLt matmul operations might have computed the output incorrectly under the following conditions: the data type of matrices A and B is FP8, the data type of matrices C and D is FP32, FP16, or BF16, the beta value is 1.0, the C and D matrices are the same, the epilogue contains GELU activation function. * When an application compiled with cuBLASLt from CUDA Toolkit 12.2 update 1 or earlier runs with cuBLASLt from CUDA Toolkit 12.2 update 2 or CUDA Toolkit 12.3, matrix multiply descriptors initialized using cublasLtMatmulDescInit() sometimes did not respect attribute changes using cublasLtMatmulDescSetAttribute(). When an application compiled with cuBLASLt from CUDA Toolkit 12.2 update 1 or earlier runs with cuBLASLt from CUDA Toolkit 12.2 update 2 or CUDA Toolkit 12.3, matrix multiply descriptors initialized using cublasLtMatmulDescInit() sometimes did not respect attribute changes using cublasLtMatmulDescSetAttribute(). * Fixed creation of cuBLAS or cuBLASLt handles on Hopper GPUs under the Multi-Process Service (MPS). Fixed creation of cuBLAS or cuBLASLt handles on Hopper GPUs under the Multi-Process Service (MPS). * cublasLtMatmul with K equals 1 and epilogue CUBLASLT_EPILOGUE_BGRAD{A,B} might have returned incorrect results for the bias gradient. cublasLtMatmul with K equals 1 and epilogue CUBLASLT_EPILOGUE_BGRAD{A,B} might have returned incorrect results for the bias gradient. 2.1.6. CUBLAS: RELEASE 12.3\uf0c1 * New Features Improved performance on NVIDIA L40S Ada GPUs. New Features * Improved performance on NVIDIA L40S Ada GPUs. Improved performance on NVIDIA L40S Ada GPUs. * Known Issues cuBLASLt matmul operations may compute the output incorrectly under the following conditions: the data type of matrices A and B is FP8, the data type of matrices C and D is FP32, FP16, or BF16, the beta value is 1.0, the C and D matrices are the same, the epilogue contains GELU activation function. When an application compiled with cuBLASLt from CUDA Toolkit 12.2 update 1 or earlier runs with cuBLASLt from CUDA Toolkit 12.2 update 2 or later, matrix multiply descriptors initialized using cublasLtMatmulDescInit() may not respect attribute changes using cublasLtMatmulDescSetAttribute(). To workaround this issue, create the matrix multiply descriptor using cublasLtMatmulDescCreate() instead of cublasLtMatmulDescInit(). This will be fixed in an upcoming release. Known Issues * cuBLASLt matmul operations may compute the output incorrectly under the following conditions: the data type of matrices A and B is FP8, the data type of matrices C and D is FP32, FP16, or BF16, the beta value is 1.0, the C and D matrices are the same, the epilogue contains GELU activation function. cuBLASLt matmul operations may compute the output incorrectly under the following conditions: the data type of matrices A and B is FP8, the data type of matrices C and D is FP32, FP16, or BF16, the beta value is 1.0, the C and D matrices are the same, the epilogue contains GELU activation function. * When an application compiled with cuBLASLt from CUDA Toolkit 12.2 update 1 or earlier runs with cuBLASLt from CUDA Toolkit 12.2 update 2 or later, matrix multiply descriptors initialized using cublasLtMatmulDescInit() may not respect attribute changes using cublasLtMatmulDescSetAttribute(). To workaround this issue, create the matrix multiply descriptor using cublasLtMatmulDescCreate() instead of cublasLtMatmulDescInit(). This will be fixed in an upcoming release. When an application compiled with cuBLASLt from CUDA Toolkit 12.2 update 1 or earlier runs with cuBLASLt from CUDA Toolkit 12.2 update 2 or later, matrix multiply descriptors initialized using cublasLtMatmulDescInit() may not respect attribute changes using cublasLtMatmulDescSetAttribute(). To workaround this issue, create the matrix multiply descriptor using cublasLtMatmulDescCreate() instead of cublasLtMatmulDescInit(). This will be fixed in an upcoming release. 2.1.7. CUBLAS: RELEASE 12.2 UPDATE 2\uf0c1 * New Features cuBLASLt will now attempt to decompose problems that cannot be run by a single gemm kernel. It does this by partitioning the problem into smaller chunks and executing the gemm kernel multiple times. This improves functional coverage for very large m, n, or batch size cases and makes the transition from the cuBLAS API to the cuBLASLt API more reliable. New Features * cuBLASLt will now attempt to decompose problems that cannot be run by a single gemm kernel. It does this by partitioning the problem into smaller chunks and executing the gemm kernel multiple times. This improves functional coverage for very large m, n, or batch size cases and makes the transition from the cuBLAS API to the cuBLASLt API more reliable. cuBLASLt will now attempt to decompose problems that cannot be run by a single gemm kernel. It does this by partitioning the problem into smaller chunks and executing the gemm kernel multiple times. This improves functional coverage for very large m, n, or batch size cases and makes the transition from the cuBLAS API to the cuBLASLt API more reliable. * Known Issues cuBLASLt matmul operations may compute the output incorrectly under the following conditions: the data type of matrices A and B is FP8, the data type of matrices C and D is FP32, FP16, or BF16, the beta value is 1.0, the C and D matrices are the same, the epilogue contains GELU activation function. Known Issues * cuBLASLt matmul operations may compute the output incorrectly under the following conditions: the data type of matrices A and B is FP8, the data type of matrices C and D is FP32, FP16, or BF16, the beta value is 1.0, the C and D matrices are the same, the epilogue contains GELU activation function. cuBLASLt matmul operations may compute the output incorrectly under the following conditions: the data type of matrices A and B is FP8, the data type of matrices C and D is FP32, FP16, or BF16, the beta value is 1.0, the C and D matrices are the same, the epilogue contains GELU activation function. 2.1.8. CUBLAS: RELEASE 12.2\uf0c1 * Known Issues cuBLAS initialization fails on Hopper architecture GPUs when MPS is in use with CUDA_MPS_ACTIVE_THREAD_PERCENTAGE set to a value less than 100%.'], 'section_id': 'cuda-libraries', 'relevance_score': 0.9952533}, page_content='2. CUDA LIBRARIES\uf0c1 This section covers CUDA Libraries release notes for 12.x releases. * CUDA Math Libraries toolchain uses C++11 features, and a C++11-compatible standard library (libstdc++ >= 20150422) is required on the host. CUDA Math Libraries toolchain uses C++11 features, and a C++11-compatible standard library (libstdc++ >= 20150422) is required on the host. 2.1. CUBLAS LIBRARY\uf0c1 2.1.1. CUBLAS: RELEASE 12.5 UPDATE 1\uf0c1 * New Features Performance improvement to matrix multiplication targeting large language models, specifically for small batch sizes on Hopper GPUs. New Features * Performance improvement to matrix multiplication targeting large language models, specifically for small batch sizes on Hopper GPUs. Performance improvement to matrix multiplication targeting large language models, specifically for small batch sizes on Hopper GPUs. * Known Issues The bias epilogue (without ReLU or GeLU) may be not supported on Hopper GPUs for strided batch cases. A workaround is to implement batching manually. This will be fixed in a future release. cublasGemmGroupedBatchedEx and cublas<t>gemmGroupedBatched have large CPU overheads. This will be addressed in an upcoming release. Known Issues * The bias epilogue (without ReLU or GeLU) may be not supported on Hopper GPUs for strided batch cases. A workaround is to implement batching manually. This will be fixed in a future release. The bias epilogue (without ReLU or GeLU) may be not supported on Hopper GPUs for strided batch cases. A workaround is to implement batching manually. This will be fixed in a future release. * cublasGemmGroupedBatchedEx and cublas<t>gemmGroupedBatched have large CPU overheads. This will be addressed in an upcoming release. cublasGemmGroupedBatchedEx and cublas<t>gemmGroupedBatched have large CPU overheads. This will be addressed in an upcoming release. * Resolved Issues Under rare circumstances, executing SYMM/HEMM concurrently with GEMM on Hopper GPUs might have caused race conditions in the host code, which could lead to an Illegal Memory Access CUDA error. cublasLtMatmul could produce an Illegal Instruction CUDA error on Pascal GPUs under the following conditions: batch is greater than 1, and beta is not equal to 0, and the computations are out-of-place (C != D). Resolved Issues * Under rare circumstances, executing SYMM/HEMM concurrently with GEMM on Hopper GPUs might have caused race conditions in the host code, which could lead to an Illegal Memory Access CUDA error. Under rare circumstances, executing SYMM/HEMM concurrently with GEMM on Hopper GPUs might have caused race conditions in the host code, which could lead to an Illegal Memory Access CUDA error. * cublasLtMatmul could produce an Illegal Instruction CUDA error on Pascal GPUs under the following conditions: batch is greater than 1, and beta is not equal to 0, and the computations are out-of-place (C != D). cublasLtMatmul could produce an Illegal Instruction CUDA error on Pascal GPUs under the following conditions: batch is greater than 1, and beta is not equal to 0, and the computations are out-of-place (C != D). 2.1.2. CUBLAS: RELEASE 12.5\uf0c1 * New Features cuBLAS adds an experimental API to support mixed precision grouped batched GEMMs. This enables grouped batched GEMMs with FP16 or BF16 inputs/outputs with the FP32 compute type. Refer to cublasGemmGroupedBatchedEx for more details. New Features * cuBLAS adds an experimental API to support mixed precision grouped batched GEMMs. This enables grouped batched GEMMs with FP16 or BF16 inputs/outputs with the FP32 compute type. Refer to cublasGemmGroupedBatchedEx for more details. cuBLAS adds an experimental API to support mixed precision grouped batched GEMMs. This enables grouped batched GEMMs with FP16 or BF16 inputs/outputs with the FP32 compute type. Refer to cublasGemmGroupedBatchedEx for more details. * Known Issues cublasLtMatmul ignores inputs to CUBLASLT_MATMUL_DESC_D_SCALE_POINTER and CUBLASLT_MATMUL_DESC_EPILOGUE_AUX_SCALE_POINTER if the elements of the respective matrix are not of FP8 types. Known Issues * cublasLtMatmul ignores inputs to CUBLASLT_MATMUL_DESC_D_SCALE_POINTER and CUBLASLT_MATMUL_DESC_EPILOGUE_AUX_SCALE_POINTER if the elements of the respective matrix are not of FP8 types. cublasLtMatmul ignores inputs to CUBLASLT_MATMUL_DESC_D_SCALE_POINTER and CUBLASLT_MATMUL_DESC_EPILOGUE_AUX_SCALE_POINTER if the elements of the respective matrix are not of FP8 types. * Resolved Issues cublasLtMatmul ignored the mismatch between the provided scale type and the implied by the documentation, assuming the latter. For instance, an unsupported configuration of cublasLtMatmul with the scale type being FP32 and all other types being FP16 would run with the implicit assumption that the scale type is FP16 and produce incorrect results. cuBLAS SYMV failed for large n dimension: 131072 and above for ssymv, 92673 and above for csymv and dsymv, and 65536 and above for zsymv. Resolved Issues * cublasLtMatmul ignored the mismatch between the provided scale type and the implied by the documentation, assuming the latter. For instance, an unsupported configuration of cublasLtMatmul with the scale type being FP32 and all other types being FP16 would run with the implicit assumption that the scale type is FP16 and produce incorrect results. cublasLtMatmul ignored the mismatch between the provided scale type and the implied by the documentation, assuming the latter. For instance, an unsupported configuration of cublasLtMatmul with the scale type being FP32 and all other types being FP16 would run with the implicit assumption that the scale type is FP16 and produce incorrect results. * cuBLAS SYMV failed for large n dimension: 131072 and above for ssymv, 92673 and above for csymv and dsymv, and 65536 and above for zsymv. cuBLAS SYMV failed for large n dimension: 131072 and above for ssymv, 92673 and above for csymv and dsymv, and 65536 and above for zsymv. 2.1.3. CUBLAS: RELEASE 12.4 UPDATE 1\uf0c1 * Known Issues Setting a cuBLAS handle stream to cudaStreamPerThread and setting the workspace via cublasSetWorkspace will cause any subsequent cublasSetWorkspace calls to fail. This will be fixed in an upcoming release. cublasLtMatmul ignores mismatches between the provided scale type and the scale type implied by the documentation and assumes the latter. For example, an unsupported configuration of cublasLtMatmul with the scale type being FP32 and all other types being FP16 would run with the implicit assumption that the scale type is FP16 which can produce incorrect results. This will be fixed in an upcoming release. Known Issues * Setting a cuBLAS handle stream to cudaStreamPerThread and setting the workspace via cublasSetWorkspace will cause any subsequent cublasSetWorkspace calls to fail. This will be fixed in an upcoming release. Setting a cuBLAS handle stream to cudaStreamPerThread and setting the workspace via cublasSetWorkspace will cause any subsequent cublasSetWorkspace calls to fail. This will be fixed in an upcoming release. * cublasLtMatmul ignores mismatches between the provided scale type and the scale type implied by the documentation and assumes the latter. For example, an unsupported configuration of cublasLtMatmul with the scale type being FP32 and all other types being FP16 would run with the implicit assumption that the scale type is FP16 which can produce incorrect results. This will be fixed in an upcoming release. cublasLtMatmul ignores mismatches between the provided scale type and the scale type implied by the documentation and assumes the latter. For example, an unsupported configuration of cublasLtMatmul with the scale type being FP32 and all other types being FP16 would run with the implicit assumption that the scale type is FP16 which can produce incorrect results. This will be fixed in an upcoming release. * Resolved Issues cublasLtMatmul ignored the CUBLASLT_MATMUL_DESC_AMAX_D_POINTER for unsupported configurations instead of returning an error. In particular, computing absolute maximum of D is currently supported only for FP8 Matmul when the output data type is also FP8 (CUDA_R_8F_E4M3 or CUDA_R_8F_E5M2). Reduced host-side overheads for some of the cuBLASLt APIs: cublasLtMatmul(), cublasLtMatmulAlgoCheck(), and cublasLtMatmulAlgoGetHeuristic(). The issue was introduced in CUDA Toolkit 12.4. cublasLtMatmul() and cublasLtMatmulAlgoGetHeuristic() could have resulted in floating point exceptions (FPE) on some Hopper-based GPUs, including Multi-Instance GPU (MIG). The issue was introduced in cuBLAS 11.8. Resolved Issues'),
    Document(metadata={'source_link': 'https://docs.nvidia.com/cuda/cuda-toolkit-release-notes/index.html', 'prechunk': ['', ''], 'postchunk': ['CUDA 11.5 Update 1 >=495.29.05 >=496.13 CUDA 11.5 GA >=495.29.05 >=496.04 CUDA 11.4 Update 4 >=470.82.01 >=472.50 CUDA 11.4 Update 3 >=470.82.01 >=472.50 CUDA 11.4 Update 2 >=470.57.02 >=471.41 CUDA 11.4 Update 1 >=470.57.02 >=471.41 CUDA 11.4.0 GA >=470.42.01 >=471.11 CUDA 11.3.1 Update 1 >=465.19.01 >=465.89 CUDA 11.3.0 GA >=465.19.01 >=465.89 CUDA 11.2.2 Update 2 >=460.32.03 >=461.33 CUDA 11.2.1 Update 1 >=460.32.03 >=461.09 CUDA 11.2.0 GA >=460.27.03 >=460.82 CUDA 11.1.1 Update 1 >=455.32 >=456.81 CUDA 11.1 GA >=455.23 >=456.38 CUDA 11.0.3 Update 1 >= 450.51.06 >= 451.82 CUDA 11.0.2 GA >= 450.51.05 >= 451.48 CUDA 11.0.1 RC >= 450.36.06 >= 451.22 CUDA 10.2.89 >= 440.33 >= 441.22 CUDA 10.1 (10.1.105 general release, and updates) >= 418.39 >= 418.96 CUDA 10.0.130 >= 410.48 >= 411.31 CUDA 9.2 (9.2.148 Update 1) >= 396.37 >= 398.26 CUDA 9.2 (9.2.88) >= 396.26 >= 397.44 CUDA 9.1 (9.1.85) >= 390.46 >= 391.29 CUDA 9.0 (9.0.76) >= 384.81 >= 385.54 CUDA 8.0 (8.0.61 GA2) >= 375.26 >= 376.51 CUDA 8.0 (8.0.44) >= 367.48 >= 369.30 CUDA 7.5 (7.5.16) >= 352.31 >= 353.66 CUDA 7.0 (7.0.28) >= 346.46 >= 347.62 For convenience, the NVIDIA driver is installed as part of the CUDA Toolkit installation. Note that this driver is for development purposes and is not recommended for use in production with Tesla GPUs. For running CUDA applications in production with Tesla GPUs, it is recommended to download the latest driver for Tesla GPUs from the NVIDIA driver downloads site at https://www.nvidia.com/drivers. During the installation of the CUDA Toolkit, the installation of the NVIDIA driver may be skipped on Windows (when using the interactive or silent installation) or on Linux (by using meta packages). For more information on customizing the install process on Windows, see https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html#install-cuda-software. For meta packages on Linux, see https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html#package-manager-metas. 1.2. NEW FEATURES\uf0c1 This section lists new general CUDA and CUDA compilers features. 1.2.1. GENERAL CUDA\uf0c1 * In an upcoming CUDA release the NVIDIA Open GPU kernel module flavor will be the default and recommended installation option. End-users with Maxwell, Pascal, or Volta GPUs may need to take action to install the NVIDIA proprietary kernel modules. In an upcoming CUDA release the NVIDIA Open GPU kernel module flavor will be the default and recommended installation option. End-users with Maxwell, Pascal, or Volta GPUs may need to take action to install the NVIDIA proprietary kernel modules. * MPS (Multi-process service) is now supported on L4T and embedded-Linux Tegra platforms. More details can be found here. MPS (Multi-process service) is now supported on L4T and embedded-Linux Tegra platforms. More details can be found here. 1.2.2. CUDA COMPILER\uf0c1 * For changes to PTX, refer to https://docs.nvidia.com/cuda/parallel-thread-execution/#ptx-isa-version-8-5. For changes to PTX, refer to https://docs.nvidia.com/cuda/parallel-thread-execution/#ptx-isa-version-8-5. 1.2.3. CUDA DEVELOPER TOOLS\uf0c1 * For changes to nvprof and Visual Profiler, see the changelog. For changes to nvprof and Visual Profiler, see the changelog. * For new features, improvements, and bug fixes in Nsight Systems, see the changelog. For new features, improvements, and bug fixes in Nsight Systems, see the changelog. * For new features, improvements, and bug fixes in Nsight Visual Studio Edition, see the changelog. For new features, improvements, and bug fixes in Nsight Visual Studio Edition, see the changelog. * For new features, improvements, and bug fixes in CUPTI, see the changelog. For new features, improvements, and bug fixes in CUPTI, see the changelog. * For new features, improvements, and bug fixes in Nsight Compute, see the changelog. For new features, improvements, and bug fixes in Nsight Compute, see the changelog. * For new features, improvements, and bug fixes in Compute Sanitizer, see the changelog. For new features, improvements, and bug fixes in Compute Sanitizer, see the changelog. * For new features, improvements, and bug fixes in CUDA-GDB, see the changelog. For new features, improvements, and bug fixes in CUDA-GDB, see the changelog. 1.3. RESOLVED ISSUES\uf0c1 1.3.1. CUDA COMPILER\uf0c1 * Resolved an issue found when trying sm89 ptx of FP8 gemm kernel compiled by 12.4 when run on an sm90 device. Resolved an issue found when trying sm89 ptx of FP8 gemm kernel compiled by 12.4 when run on an sm90 device. * Resolved an issue in which nvcc failed to compile any CUDA code when specifying C++20 with CUDA 12.5 and Visual Studio 2022 17.10.0. Also added a new environment variable NVCC_REPORT_ALLERROR to emit error messages if the error is coming from a system header, instead of aborting the compiler. Resolved an issue in which nvcc failed to compile any CUDA code when specifying C++20 with CUDA 12.5 and Visual Studio 2022 17.10.0. Also added a new environment variable NVCC_REPORT_ALLERROR to emit error messages if the error is coming from a system header, instead of aborting the compiler. * Resolved a compiler issue that caused different results when compiling with the -G flag than without the flag. Resolved a compiler issue that caused different results when compiling with the -G flag than without the flag. * Fixed the incorrect control flow transformation in the compiler caused by optimizations applied to multi-block loops. Fixed the incorrect control flow transformation in the compiler caused by optimizations applied to multi-block loops. * Resolved issues seen when compiling cuBLASDx device functions, in some conditions leading to “Misaligned shared or local address”. Resolved issues seen when compiling cuBLASDx device functions, in some conditions leading to “Misaligned shared or local address”. * Fix to correct the calculation of write-after-read hazard latency. Fix to correct the calculation of write-after-read hazard latency. 1.4. KNOWN ISSUES AND LIMITATIONS\uf0c1 * Runfile will not be supported for Amazon Linux 2023. Runfile will not be supported for Amazon Linux 2023. * Confidential Computing is not supported on CUDA 12.5. Please continue to use CUDA 12.4 and drivers r550.xx to use these features. Confidential Computing is not supported on CUDA 12.5. Please continue to use CUDA 12.4 and drivers r550.xx to use these features. * Launching Cooperative Group kernels with MPS is not supported on Tegra platforms. Launching Cooperative Group kernels with MPS is not supported on Tegra platforms. 1.5. DEPRECATED OR DROPPED FEATURES\uf0c1 Features deprecated in the current release of the CUDA software still work in the current release, but their documentation may have been removed, and they will become officially unsupported in a future release. We recommend that developers employ alternative solutions to these features in their software. 1.5.1. DEPRECATED OR DROPPED ARCHITECTURES\uf0c1 * NVIDIA CUDA support for the PowerPC architecture is removed in CUDA 12.5. NVIDIA CUDA support for the PowerPC architecture is removed in CUDA 12.5. 1.5.2. DEPRECATED OPERATING SYSTEMS\uf0c1 * NVIDIA CUDA support for Red Hat Enterprise Linux 7 and CentOS 7 is removed in CUDA 12.5. NVIDIA CUDA support for Red Hat Enterprise Linux 7 and CentOS 7 is removed in CUDA 12.5. * CUDA 12.5 is the last release to support Debian 10.', 'CUDA 12.5 is the last release to support Debian 10. * Support for Microsoft Windows 10 21H2 and Microsoft Windows 10 21H2 (SV1) is deprecated. Support for Microsoft Windows 10 21H2 and Microsoft Windows 10 21H2 (SV1) is deprecated. 1.5.3. DEPRECATED TOOLCHAINS\uf0c1 CUDA Toolkit 12.4 deprecated support for the following host compilers: * Microsoft Visual C/C++ (MSVC) 2017 Microsoft Visual C/C++ (MSVC) 2017 * All GCC versions prior to GCC 7.3 All GCC versions prior to GCC 7.3 1.5.4. CUDA TOOLS\uf0c1 * Support for the macOS host client of CUDA-GDB is deprecated. It will be dropped in an upcoming release. Support for the macOS host client of CUDA-GDB is deprecated. It will be dropped in an upcoming release.'], 'section_id': 'cuda-12-5-update-1-release-notes', 'relevance_score': 0.98736477}, page_content='1. CUDA 12.5 UPDATE 1 RELEASE NOTES\uf0c1 The release notes for the NVIDIA® CUDA® Toolkit can be found online at https://docs.nvidia.com/cuda/cuda-toolkit-release-notes/index.html. Note The release notes have been reorganized into two major sections: the general CUDA release notes, and the CUDA libraries release notes including historical information for 12.x releases. 1.1. CUDA TOOLKIT MAJOR COMPONENT VERSIONS\uf0c1 Starting with CUDA 11, the various components in the toolkit are versioned independently. For CUDA 12.5 Update 1, the table below indicates the versions: Component Name Version Information Supported Architectures Supported Platforms CUDA C++ Core Compute Libraries Thrust 2.4.0 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows CUB 2.4.0 libcu++ 2.4.0 Cooperative Groups 12.5.82 CUDA Compatibility 12.5.36505571 aarch64-jetson Linux CUDA Runtime (cudart) 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL cuobjdump 12.5.39 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows CUPTI 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA cuxxfilt (demangler) 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows CUDA Demo Suite 12.5.82 x86_64 Linux, Windows CUDA GDB 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, WSL CUDA Nsight Eclipse Plugin 12.5.82 x86_64 Linux CUDA NVCC 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA nvdisasm 12.5.39 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows CUDA NVML Headers 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA nvprof 12.5.82 x86_64 Linux, Windows CUDA nvprune 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA NVRTC 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL NVTX 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA NVVP 12.5.82 x86_64, Linux, Windows CUDA OpenCL 12.5.39 x86_64 Linux, Windows CUDA Profiler API 12.5.39 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA Compute Sanitizer API 12.5.81 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA cuBLAS 12.5.3.2 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL cuDLA 12.5.82 aarch64-jetson Linux CUDA cuFFT 11.2.3.61 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA cuFile 1.10.1.7 x86_64, arm64-sbsa, aarch64-jetson Linux CUDA cuRAND 10.3.6.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA cuSOLVER 11.6.3.83 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA cuSPARSE 12.5.1.3 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA NPP 12.3.0.159 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA nvFatbin 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA nvJitLink 12.5.82 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL CUDA nvJPEG 12.3.2.81 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL Nsight Compute 2024.2.1.2 x86_64, arm64-sbsa, aarch64-jetson Linux, Windows, WSL (Windows 11) Nsight Systems 2024.2.3.38 x86_64, arm64-sbsa, Linux, Windows, WSL Nsight Visual Studio Edition (VSE) 2024.2.1.24155 x86_64 (Windows) Windows nvidia_fs1 2.20.6 x86_64, arm64-sbsa, aarch64-jetson Linux Visual Studio Integration 12.5.82 x86_64 (Windows) Windows NVIDIA Linux Driver 555.42.06 x86_64, arm64-sbsa Linux NVIDIA Windows Driver 555.85 x86_64 (Windows) Windows, WSL Running a CUDA application requires the system with at least one CUDA capable GPU and a driver that is compatible with the CUDA Toolkit. See Table 3. For more information various GPU products that are CUDA capable, visit https://developer.nvidia.com/cuda-gpus. Each release of the CUDA Toolkit requires a minimum version of the CUDA driver. The CUDA driver is backward compatible, meaning that applications compiled against a particular version of the CUDA will continue to work on subsequent (later) driver releases. More information on compatibility can be found at https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#cuda-compatibility-and-upgrades. Note: Starting with CUDA 11.0, the toolkit components are individually versioned, and the toolkit itself is versioned as shown in the table below. The minimum required driver version for CUDA minor version compatibility is shown below. CUDA minor version compatibility is described in detail in https://docs.nvidia.com/deploy/cuda-compatibility/index.html CUDA Toolkit Minimum Required Driver Version for CUDA Minor Version Compatibility* Linux x86_64 Driver Version Windows x86_64 Driver Version CUDA 12.x >=525.60.13 >=528.33 CUDA 11.8.x CUDA 11.7.x CUDA 11.6.x CUDA 11.5.x CUDA 11.4.x CUDA 11.3.x CUDA 11.2.x CUDA 11.1.x >=450.80.02 >=452.39 CUDA 11.0 (11.0.3) >=450.36.06** >=451.22** * Using a Minimum Required Version that is different from Toolkit Driver Version could be allowed in compatibility mode – please read the CUDA Compatibility Guide for details. ** CUDA 11.0 was released with an earlier driver version, but by upgrading to Tesla Recommended Drivers 450.80.02 (Linux) / 452.39 (Windows), minor version compatibility is possible across the CUDA 11.x family of toolkits. The version of the development NVIDIA GPU Driver packaged in each CUDA Toolkit release is shown below. CUDA Toolkit Toolkit Driver Version Linux x86_64 Driver Version Windows x86_64 Driver Version CUDA 12.5 Update 1 >=555.42.06 >=555.85 CUDA 12.5 GA >=555.42.02 >=555.85 CUDA 12.4 Update 1 >=550.54.15 >=551.78 CUDA 12.4 GA >=550.54.14 >=551.61 CUDA 12.3 Update 1 >=545.23.08 >=546.12 CUDA 12.3 GA >=545.23.06 >=545.84 CUDA 12.2 Update 2 >=535.104.05 >=537.13 CUDA 12.2 Update 1 >=535.86.09 >=536.67 CUDA 12.2 GA >=535.54.03 >=536.25 CUDA 12.1 Update 1 >=530.30.02 >=531.14 CUDA 12.1 GA >=530.30.02 >=531.14 CUDA 12.0 Update 1 >=525.85.12 >=528.33 CUDA 12.0 GA >=525.60.13 >=527.41 CUDA 11.8 GA >=520.61.05 >=520.06 CUDA 11.7 Update 1 >=515.48.07 >=516.31 CUDA 11.7 GA >=515.43.04 >=516.01 CUDA 11.6 Update 2 >=510.47.03 >=511.65 CUDA 11.6 Update 1 >=510.47.03 >=511.65 CUDA 11.6 GA >=510.39.01 >=511.23 CUDA 11.5 Update 2 >=495.29.05 >=496.13'),
    Document(metadata={'source_link': 'https://docs.nvidia.com/cuda/cuda-toolkit-release-notes/index.html', 'prechunk': ['* cusparseSpSM_bufferSize() could ask slightly less memory than needed. cusparseSpSM_bufferSize() could ask slightly less memory than needed. * cusparseSpMV() now checks the validity of the buffer pointer only when it is strictly needed. cusparseSpMV() now checks the validity of the buffer pointer only when it is strictly needed. * Deprecations Several legacy APIs have been officially deprecated. A compile-time warning has been added to all of them. Deprecations * Several legacy APIs have been officially deprecated. A compile-time warning has been added to all of them. Several legacy APIs have been officially deprecated. A compile-time warning has been added to all of them. 2.4.7. CUSPARSE: RELEASE 12.1 UPDATE 1\uf0c1 * New Features Introduced Block Sparse Row (BSR) sparse matrix storage for the Generic APIs with support for SDDMM routine (cusparseSDDMM). Introduced Sliced Ellpack (SELL) sparse matrix storage format for the Generic APIs with support for sparse matrix-vector multiplication (cusparseSpMV) and triangular solver with a single right-hand side (cusparseSpSV). Added a new API call (cusparseSpSV_updateMatrix) to update matrix values and/or the matrix diagonal in the sparse triangular solver with a single right-hand side after the analysis step. New Features * Introduced Block Sparse Row (BSR) sparse matrix storage for the Generic APIs with support for SDDMM routine (cusparseSDDMM). Introduced Block Sparse Row (BSR) sparse matrix storage for the Generic APIs with support for SDDMM routine (cusparseSDDMM). * Introduced Sliced Ellpack (SELL) sparse matrix storage format for the Generic APIs with support for sparse matrix-vector multiplication (cusparseSpMV) and triangular solver with a single right-hand side (cusparseSpSV). Introduced Sliced Ellpack (SELL) sparse matrix storage format for the Generic APIs with support for sparse matrix-vector multiplication (cusparseSpMV) and triangular solver with a single right-hand side (cusparseSpSV). * Added a new API call (cusparseSpSV_updateMatrix) to update matrix values and/or the matrix diagonal in the sparse triangular solver with a single right-hand side after the analysis step. Added a new API call (cusparseSpSV_updateMatrix) to update matrix values and/or the matrix diagonal in the sparse triangular solver with a single right-hand side after the analysis step. 2.4.8. CUSPARSE: RELEASE 12.0 UPDATE 1\uf0c1 * New Features cusparseSDDMM() now supports mixed precision computation. Improved cusparseSpMM() alg2 mixed-precision performance on some matrices on NVIDIA Ampere architecture GPUs. Improved cusparseSpMV() performance with a new load balancing algorithm. cusparseSpSV() and cusparseSpSM() now support in-place computation, namely the output and input vectors/matrices have the same memory address. New Features * cusparseSDDMM() now supports mixed precision computation. cusparseSDDMM() now supports mixed precision computation. * Improved cusparseSpMM() alg2 mixed-precision performance on some matrices on NVIDIA Ampere architecture GPUs. Improved cusparseSpMM() alg2 mixed-precision performance on some matrices on NVIDIA Ampere architecture GPUs. * Improved cusparseSpMV() performance with a new load balancing algorithm. Improved cusparseSpMV() performance with a new load balancing algorithm. * cusparseSpSV() and cusparseSpSM() now support in-place computation, namely the output and input vectors/matrices have the same memory address. cusparseSpSV() and cusparseSpSM() now support in-place computation, namely the output and input vectors/matrices have the same memory address. * Resolved Issues cusparseSpSM() could produce wrong results if the leading dimension (ld) of the RHS matrix is greater than the number of columns/rows. Resolved Issues * cusparseSpSM() could produce wrong results if the leading dimension (ld) of the RHS matrix is greater than the number of columns/rows. cusparseSpSM() could produce wrong results if the leading dimension (ld) of the RHS matrix is greater than the number of columns/rows. 2.4.9. CUSPARSE: RELEASE 12.0\uf0c1 * New Features JIT LTO functionalities (cusparseSpMMOp()) switched from driver to nvJitLto library. Starting from CUDA 12.0 the user needs to link to libnvJitLto.so, see cuSPARSE documentation. JIT LTO performance has also been improved for cusparseSpMMOpPlan(). Introduced const descriptors for the Generic APIs, for example, cusparseConstSpVecGet(). Now the Generic APIs interface clearly declares when a descriptor and its data are modified by the cuSPARSE functions. Added two new algorithms to cusparseSpGEMM() with lower memory utilization. The first algorithm computes a strict bound on the number of intermediate product, while the second one allows partitioning the computation in chunks. Added int8_t support to cusparseGather(), cusparseScatter(), and cusparseCsr2cscEx2(). Improved cusparseSpSV() performance for both the analysis and the solving phases. Improved cusparseSpSM() performance for both the analysis and the solving phases. Improved cusparseSDDMM() performance and added support for batch computation. Improved cusparseCsr2cscEx2() performance. New Features * JIT LTO functionalities (cusparseSpMMOp()) switched from driver to nvJitLto library. Starting from CUDA 12.0 the user needs to link to libnvJitLto.so, see cuSPARSE documentation. JIT LTO performance has also been improved for cusparseSpMMOpPlan(). JIT LTO functionalities (cusparseSpMMOp()) switched from driver to nvJitLto library. Starting from CUDA 12.0 the user needs to link to libnvJitLto.so, see cuSPARSE documentation. JIT LTO performance has also been improved for cusparseSpMMOpPlan(). * Introduced const descriptors for the Generic APIs, for example, cusparseConstSpVecGet(). Now the Generic APIs interface clearly declares when a descriptor and its data are modified by the cuSPARSE functions. Introduced const descriptors for the Generic APIs, for example, cusparseConstSpVecGet(). Now the Generic APIs interface clearly declares when a descriptor and its data are modified by the cuSPARSE functions. * Added two new algorithms to cusparseSpGEMM() with lower memory utilization. The first algorithm computes a strict bound on the number of intermediate product, while the second one allows partitioning the computation in chunks. Added two new algorithms to cusparseSpGEMM() with lower memory utilization. The first algorithm computes a strict bound on the number of intermediate product, while the second one allows partitioning the computation in chunks. * Added int8_t support to cusparseGather(), cusparseScatter(), and cusparseCsr2cscEx2(). Added int8_t support to cusparseGather(), cusparseScatter(), and cusparseCsr2cscEx2(). * Improved cusparseSpSV() performance for both the analysis and the solving phases. Improved cusparseSpSV() performance for both the analysis and the solving phases. * Improved cusparseSpSM() performance for both the analysis and the solving phases. Improved cusparseSpSM() performance for both the analysis and the solving phases. * Improved cusparseSDDMM() performance and added support for batch computation. Improved cusparseSDDMM() performance and added support for batch computation. * Improved cusparseCsr2cscEx2() performance. Improved cusparseCsr2cscEx2() performance. * Resolved Issues cusparseSpSV() and cusparseSpSM() could produce wrong results. cusparseDnMatGetStridedBatch() did not accept batchStride == 0. Resolved Issues * cusparseSpSV() and cusparseSpSM() could produce wrong results. cusparseSpSV() and cusparseSpSM() could produce wrong results. * cusparseDnMatGetStridedBatch() did not accept batchStride == 0. cusparseDnMatGetStridedBatch() did not accept batchStride == 0. * Deprecations Removed deprecated CUDA 11.x APIs, enumerators, and descriptors. Deprecations * Removed deprecated CUDA 11.x APIs, enumerators, and descriptors. Removed deprecated CUDA 11.x APIs, enumerators, and descriptors. 2.5. MATH LIBRARY\uf0c1 2.5.1. CUDA MATH: RELEASE 12.5\uf0c1 * Known Issues As a result of ongoing testing we updated the interval bounds in which double precision lgamma() function may experience greater than the documented 4 ulp accuracy loss. New interval shall read (-23.0001; -2.2637). This finding is applicable to CUDA 12.5 and all previous versions. Known Issues * As a result of ongoing testing we updated the interval bounds in which double precision lgamma() function may experience greater than the documented 4 ulp accuracy loss. New interval shall read (-23.0001; -2.2637). This finding is applicable to CUDA 12.5 and all previous versions. As a result of ongoing testing we updated the interval bounds in which double precision lgamma() function may experience greater than the documented 4 ulp accuracy loss. New interval shall read (-23.0001; -2.2637). This finding is applicable to CUDA 12.5 and all previous versions. 2.5.2. CUDA MATH:', 'RELEASE 12.4\uf0c1 * Resolved Issues Host-specific code in cuda_fp16/bf16 headers is now free from type-punning and shall work correctly in the presence of optimizations based on strict-aliasing rules. Resolved Issues * Host-specific code in cuda_fp16/bf16 headers is now free from type-punning and shall work correctly in the presence of optimizations based on strict-aliasing rules. Host-specific code in cuda_fp16/bf16 headers is now free from type-punning and shall work correctly in the presence of optimizations based on strict-aliasing rules. 2.5.3. CUDA MATH: RELEASE 12.3\uf0c1 * New Features Performance of SIMD Integer CUDA Math APIs was improved. New Features * Performance of SIMD Integer CUDA Math APIs was improved. Performance of SIMD Integer CUDA Math APIs was improved. * Resolved Issues The __hisinf() Math APIs from cuda_fp16.h and cuda_bf16.h headers were silently producing wrong results if compiled with the -std=c++20 compiler option because of an underlying nvcc compiler issue, resolved in version 12.3. Resolved Issues * The __hisinf() Math APIs from cuda_fp16.h and cuda_bf16.h headers were silently producing wrong results if compiled with the -std=c++20 compiler option because of an underlying nvcc compiler issue, resolved in version 12.3. The __hisinf() Math APIs from cuda_fp16.h and cuda_bf16.h headers were silently producing wrong results if compiled with the -std=c++20 compiler option because of an underlying nvcc compiler issue, resolved in version 12.3. * Known Issues Users of cuda_fp16.h and cuda_bf16.h headers are advised to disable host compilers strict aliasing rules based optimizations (e.g. pass -fno-strict-aliasing to host GCC compiler) as these may interfere with the type-punning idioms used in the __half, __half2, __nv_bfloat16, __nv_bfloat162 types implementations and expose the user program to undefined behavior. Note, the headers suppress GCC diagnostics through: #pragma GCC diagnostic ignored -Wstrict-aliasing. This behavior may improve in future versions of the headers. Known Issues * Users of cuda_fp16.h and cuda_bf16.h headers are advised to disable host compilers strict aliasing rules based optimizations (e.g. pass -fno-strict-aliasing to host GCC compiler) as these may interfere with the type-punning idioms used in the __half, __half2, __nv_bfloat16, __nv_bfloat162 types implementations and expose the user program to undefined behavior. Note, the headers suppress GCC diagnostics through: #pragma GCC diagnostic ignored -Wstrict-aliasing. This behavior may improve in future versions of the headers. Users of cuda_fp16.h and cuda_bf16.h headers are advised to disable host compilers strict aliasing rules based optimizations (e.g. pass -fno-strict-aliasing to host GCC compiler) as these may interfere with the type-punning idioms used in the __half, __half2, __nv_bfloat16, __nv_bfloat162 types implementations and expose the user program to undefined behavior. Note, the headers suppress GCC diagnostics through: #pragma GCC diagnostic ignored -Wstrict-aliasing. This behavior may improve in future versions of the headers. 2.5.4. CUDA MATH: RELEASE 12.2\uf0c1 * New Features CUDA Math APIs for __half and __nv_bfloat16 types received usability improvements, including host side <emulated> support for many of the arithmetic operations and conversions. __half and __nv_bfloat16 types have implicit conversions to/from integral types, which are now available with host compilers by default. These may cause build issues due to ambiguous overloads resolution. Users are advised to update their code to select proper overloads. To opt-out user may want to define the following macros (these macros will be removed in the future CUDA release): __CUDA_FP16_DISABLE_IMPLICIT_INTEGER_CONVERTS_FOR_HOST_COMPILERS__ __CUDA_BF16_DISABLE_IMPLICIT_INTEGER_CONVERTS_FOR_HOST_COMPILERS__ New Features * CUDA Math APIs for __half and __nv_bfloat16 types received usability improvements, including host side <emulated> support for many of the arithmetic operations and conversions. CUDA Math APIs for __half and __nv_bfloat16 types received usability improvements, including host side <emulated> support for many of the arithmetic operations and conversions. * __half and __nv_bfloat16 types have implicit conversions to/from integral types, which are now available with host compilers by default. These may cause build issues due to ambiguous overloads resolution. Users are advised to update their code to select proper overloads. To opt-out user may want to define the following macros (these macros will be removed in the future CUDA release): __CUDA_FP16_DISABLE_IMPLICIT_INTEGER_CONVERTS_FOR_HOST_COMPILERS__ __CUDA_BF16_DISABLE_IMPLICIT_INTEGER_CONVERTS_FOR_HOST_COMPILERS__ __half and __nv_bfloat16 types have implicit conversions to/from integral types, which are now available with host compilers by default. These may cause build issues due to ambiguous overloads resolution. Users are advised to update their code to select proper overloads. To opt-out user may want to define the following macros (these macros will be removed in the future CUDA release): * __CUDA_FP16_DISABLE_IMPLICIT_INTEGER_CONVERTS_FOR_HOST_COMPILERS__ __CUDA_FP16_DISABLE_IMPLICIT_INTEGER_CONVERTS_FOR_HOST_COMPILERS__ * __CUDA_BF16_DISABLE_IMPLICIT_INTEGER_CONVERTS_FOR_HOST_COMPILERS__ __CUDA_BF16_DISABLE_IMPLICIT_INTEGER_CONVERTS_FOR_HOST_COMPILERS__ * Resolved Issues During ongoing testing, NVIDIA identified that due to an algorithm error the results of 64-bit floating-point division in default round-to-nearest-even mode could produce spurious overflow to infinity. NVIDIA recommends that all developers requiring strict IEEE754 compliance update to CUDA Toolkit 12.2 or newer. The affected algorithm was present in both offline compilation as well as just-in-time (JIT) compilation. As JIT compilation is handled by the driver, NVIDIA recommends updating to driver version greater than or equal to R535 (R536 on Windows) when IEEE754 compliance is required and when using JIT. This is a software algorithm fix and is not tied to specific hardware. Updated the observed worst case error bounds for single precision intrinsic functions __expf(), __exp10f() and double precision functions asinh(), acosh(). Resolved Issues * During ongoing testing, NVIDIA identified that due to an algorithm error the results of 64-bit floating-point division in default round-to-nearest-even mode could produce spurious overflow to infinity. NVIDIA recommends that all developers requiring strict IEEE754 compliance update to CUDA Toolkit 12.2 or newer. The affected algorithm was present in both offline compilation as well as just-in-time (JIT) compilation. As JIT compilation is handled by the driver, NVIDIA recommends updating to driver version greater than or equal to R535 (R536 on Windows) when IEEE754 compliance is required and when using JIT. This is a software algorithm fix and is not tied to specific hardware. During ongoing testing, NVIDIA identified that due to an algorithm error the results of 64-bit floating-point division in default round-to-nearest-even mode could produce spurious overflow to infinity. NVIDIA recommends that all developers requiring strict IEEE754 compliance update to CUDA Toolkit 12.2 or newer. The affected algorithm was present in both offline compilation as well as just-in-time (JIT) compilation. As JIT compilation is handled by the driver, NVIDIA recommends updating to driver version greater than or equal to R535 (R536 on Windows) when IEEE754 compliance is required and when using JIT. This is a software algorithm fix and is not tied to specific hardware. * Updated the observed worst case error bounds for single precision intrinsic functions __expf(), __exp10f() and double precision functions asinh(), acosh(). Updated the observed worst case error bounds for single precision intrinsic functions __expf(), __exp10f() and double precision functions asinh(), acosh(). 2.5.5. CUDA MATH: RELEASE 12.1\uf0c1 * New Features Performance and accuracy improvements in atanf, acosf, asinf, sinpif, cospif, powf, erff, and tgammaf. New Features * Performance and accuracy improvements in atanf, acosf, asinf, sinpif, cospif, powf, erff, and tgammaf. Performance and accuracy improvements in atanf, acosf, asinf, sinpif, cospif, powf, erff, and tgammaf. 2.5.6. CUDA MATH: RELEASE 12.0\uf0c1 * New Features Introduced new integer/fp16/bf16 CUDA Math APIs to help expose performance benefits of new DPX instructions. Refer to https://docs.nvidia.com/cuda/cuda-math-api/index.html. New Features * Introduced new integer/fp16/bf16 CUDA Math APIs to help expose performance benefits of new DPX instructions. Refer to https://docs.nvidia.com/cuda/cuda-math-api/index.html. Introduced new integer/fp16/bf16 CUDA Math APIs to help expose performance benefits of new DPX instructions. Refer to https://docs.nvidia.com/cuda/cuda-math-api/index.html. * Known Issues Double precision inputs that cause the double precision division algorithm in the default ‘round to nearest even mode’ produce spurious overflow: an infinite result is delivered where DBL_MAX 0x7FEF_FFFF_FFFF_FFFF is expected.'], 'postchunk': ['', ''], 'section_id': 'cuda-libraries', 'relevance_score': 0.0032670058}, page_content='Affected CUDA Math APIs: __ddiv_rn(). Affected CUDA language operation: double precision / operation in the device code. Known Issues * Double precision inputs that cause the double precision division algorithm in the default ‘round to nearest even mode’ produce spurious overflow: an infinite result is delivered where DBL_MAX 0x7FEF_FFFF_FFFF_FFFF is expected. Affected CUDA Math APIs: __ddiv_rn(). Affected CUDA language operation: double precision / operation in the device code. Double precision inputs that cause the double precision division algorithm in the default ‘round to nearest even mode’ produce spurious overflow: an infinite result is delivered where DBL_MAX 0x7FEF_FFFF_FFFF_FFFF is expected. Affected CUDA Math APIs: __ddiv_rn(). Affected CUDA language operation: double precision / operation in the device code. * Deprecations All previously deprecated undocumented APIs are removed from CUDA 12.0. Deprecations * All previously deprecated undocumented APIs are removed from CUDA 12.0. All previously deprecated undocumented APIs are removed from CUDA 12.0. 2.6. NVIDIA PERFORMANCE PRIMITIVES (NPP)\uf0c1 2.6.1. NPP: RELEASE 12.4\uf0c1 * New Features Enhanced large file support with size_t. New Features * Enhanced large file support with size_t. Enhanced large file support with size_t. 2.6.2. NPP: RELEASE 12.0\uf0c1 * Deprecations Deprecating non-CTX API support from next release. Deprecations * Deprecating non-CTX API support from next release. Deprecating non-CTX API support from next release. * Resolved Issues A performance issue with the NPP ResizeSqrPixel API is now fixed and shows improved performance. Resolved Issues * A performance issue with the NPP ResizeSqrPixel API is now fixed and shows improved performance. A performance issue with the NPP ResizeSqrPixel API is now fixed and shows improved performance. 2.7. NVJPEG LIBRARY\uf0c1 2.7.1. NVJPEG: RELEASE 12.4\uf0c1 * New Features IDCT performance optimizations for single image CUDA decode. Zero Copy behavior has been changed: Setting NVJPEG_FLAGS_REDUCED_MEMORY_DECODE_ZERO_COPY flag will no longer enable NVJPEG_FLAGS_REDUCED_MEMORY_DECODE. New Features * IDCT performance optimizations for single image CUDA decode. IDCT performance optimizations for single image CUDA decode. * Zero Copy behavior has been changed: Setting NVJPEG_FLAGS_REDUCED_MEMORY_DECODE_ZERO_COPY flag will no longer enable NVJPEG_FLAGS_REDUCED_MEMORY_DECODE. Zero Copy behavior has been changed: Setting NVJPEG_FLAGS_REDUCED_MEMORY_DECODE_ZERO_COPY flag will no longer enable NVJPEG_FLAGS_REDUCED_MEMORY_DECODE. 2.7.2. NVJPEG: RELEASE 12.3 UPDATE 1\uf0c1 * New Features New APIs: nvjpegBufferPinnedResize and nvjpegBufferDeviceResize which can be used to resize pinned and device buffers before using them. New Features * New APIs: nvjpegBufferPinnedResize and nvjpegBufferDeviceResize which can be used to resize pinned and device buffers before using them. New APIs: nvjpegBufferPinnedResize and nvjpegBufferDeviceResize which can be used to resize pinned and device buffers before using them. 2.7.3. NVJPEG: RELEASE 12.2\uf0c1 * New Features Added support for JPEG Lossless decode (process 14, FO prediction). nvJPEG is now supported on L4T. New Features * Added support for JPEG Lossless decode (process 14, FO prediction). Added support for JPEG Lossless decode (process 14, FO prediction). * nvJPEG is now supported on L4T. nvJPEG is now supported on L4T. 2.7.4. NVJPEG: RELEASE 12.0\uf0c1 * New Features Immproved the GPU Memory optimisation for the nvJPEG codec. New Features * Immproved the GPU Memory optimisation for the nvJPEG codec. Immproved the GPU Memory optimisation for the nvJPEG codec. * Resolved Issues An issue that causes runtime failures when nvJPEGDecMultipleInstances was tested with a large number of threads is resolved. An issue with CMYK four component color conversion is now resolved. Resolved Issues * An issue that causes runtime failures when nvJPEGDecMultipleInstances was tested with a large number of threads is resolved. An issue that causes runtime failures when nvJPEGDecMultipleInstances was tested with a large number of threads is resolved. * An issue with CMYK four component color conversion is now resolved. An issue with CMYK four component color conversion is now resolved. * Known Issues Backend NVJPEG_BACKEND_GPU_HYBRID - Unable to handle bistreams with extra scans lengths. Known Issues * Backend NVJPEG_BACKEND_GPU_HYBRID - Unable to handle bistreams with extra scans lengths. Backend NVJPEG_BACKEND_GPU_HYBRID - Unable to handle bistreams with extra scans lengths. * Deprecations The reuse of Huffman table in Encoder (nvjpegEncoderParamsCopyHuffmanTables). Deprecations * The reuse of Huffman table in Encoder (nvjpegEncoderParamsCopyHuffmanTables). The reuse of Huffman table in Encoder (nvjpegEncoderParamsCopyHuffmanTables). Only available on select Linux distros'),
]

In [106]:
MASTER_PROMPT = """Note:
            Make sure to address the user's queries politely.
            Compose a comprehensive reply to the query using the search results given.
            1.Respond to the questions based on the given context. 
            2.In the output mention the source link, at the end of the answer by [Source: link of a source], Don't provide multiple sources if the answer is from the same source.
            3.please refrain from inventing responses nd kindly respond with "I apologize, but that falls outside of my current scope of knowledge.".
            4.Use as much detail when as possible while responding. Take a deep breath and Answer step-by-step.
            5.All answers should be in MARKDOWN (.md) Format:""" 

support_template = """ 
        {MASTER_PROMPT}
    
    CONTEXT:
    {context}

    CHAT HISTORY:
    {chat_history}

    Question: 
    {question}
    """

QA_PROMPT = PromptTemplate(
    template=support_template, input_variables=["MASTER_PROMPT", "context", "chat_history", "question"]
)

In [107]:
def calculate_cost(total_usage:Dict):
    completion_tokens = total_usage['token_usage']['completion_tokens']
    prompt_tokens = total_usage['token_usage']['prompt_tokens']

    #in $
    input_token = (prompt_tokens/1000)*0.0065
    output_token = (completion_tokens/1000)*0.0195
    total_cost = input_token+output_token
    return total_cost

In [108]:
def format_result(result:AIMessage):
    response = result.content
    response_metadata = result.response_metadata
    return response, response_metadata

In [109]:
def chatbot(question, formatted_context, retrieved_history:List):
    llm_model = initialise_llm_model(LLM_MODEL)
    history = []
    if retrieved_history:
        if len(retrieved_history)>=2:
            history = retrieved_history[-2:]
        else:
            history = retrieved_history

    rag_chain = (
        {
            "context": RunnablePassthrough(),
            "question": RunnablePassthrough(),
            "chat_history": RunnablePassthrough(),
            "MASTER_PROMPT": RunnablePassthrough()
        }
        | QA_PROMPT
        | llm_model
    )

    # rag_chain = (
    #     RunnablePassthrough.assign(context=formatted_context, question=question)  # Pass context and question
    #     | QA_PROMPT  # Pass through the QA prompt
    #     | llm_model  # Finally, pass through the LLM model
    # )
    try:
        with get_openai_callback() as cb:
            response = rag_chain.invoke({"context":formatted_context,"chat_history":history, "question": question, "MASTER_PROMPT": MASTER_PROMPT},{"callbacks": [cb]})      #{"question": question, "chat_history": history, "MASTER_PROMPT_FROM_USER" : MASTER_PROMPT_FROM_USER},
            result, token_usage = format_result(response)
            total_cost = calculate_cost(token_usage)
            return (result, total_cost)
    except Exception as e:
        return (str(e), "")

In [110]:
chat_history = [("What is cuda", "cuda is the package provided for gpu distributed programming to efficiently used GPU cores and memory"), ("Tell me about the gpu","A GPU (Graphics Processing Unit) is a specialized processor designed to accelerate graphics rendering and perform parallel processing tasks, commonly used in gaming, machine learning, and scientific computations.")]

In [111]:
def Advance_RAG_Chatbot(question):
    st_time = time.time()
    expanded_queries = Custom_Query_Exapander(question)
    combined_results = []
    for query in expanded_queries:
        output = milvus_hybrid_search(question, expr="")
        print(output)
        combined_results.extend(output)
    reranked_docs = Reranker(question, combined_results)
    formatted_context = format_docs(reranked_docs)
    response = chatbot(question, formatted_context, chat_history)
    end_time = time.time()-st_time
    return (response, end_time)

In [112]:
Response = Advance_RAG_Chatbot(QUESTION)
Response

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 81284.96it/s]
INFO:httpx:HTTP Request: POST https://api.aimlapi.com/v1/chat/completions "HTTP/1.1 201 Created"
INFO:langchain.retrievers.multi_query:Generated queries: ['What is advanced manufacturing?', '', 'What are the key technologies used in advanced manufacturing?', '', 'What are the benefits of advanced manufacturing?', '', 'How does advanced manufacturing differ from traditional manufacturing?', '', 'What industries are most impacted by advanced manufacturing?', '', 'What are some examples of advanced manufacturing techniques?', '', 'What are the challenges faced in implementing advanced manufacturing?', '', 'What is the future outlook for advanced manufacturing?']
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 79739.62it/s]


Hybrid Search Result: data: ['["id: 451726673013646285, distance: 0.032522473484277725, entity: {\'text\': \'In the next 10 years, you will undoubtedly see instances of new entrants being acquired by traditional original equipment manufacturers (OEMs). Both have problems that are not easy to solve. But when these two types of companies are paired together, these problems can be addressed.\\\\nBoth new entrants and traditional OEMs have strengths and weaknesses — and both have competitive advantages. Having less experience means having fewer constraints and less fear — after all, there is less to lose. But it also comes with a lack of wisdom and practical knowledge, as well as a tendency to make more mistakes along the way.\\\\nThe automotive industry has existed for over 100 years. A lot of the large OEMs have spent decades really honing how they can make their supply chain lean; build and sell cars on a large scale, at a high volume and cost-effectively; get the cars out to consumers 

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 72817.78it/s]


Hybrid Search Result: data: ['["id: 451726673013646285, distance: 0.032522473484277725, entity: {\'text\': \'In the next 10 years, you will undoubtedly see instances of new entrants being acquired by traditional original equipment manufacturers (OEMs). Both have problems that are not easy to solve. But when these two types of companies are paired together, these problems can be addressed.\\\\nBoth new entrants and traditional OEMs have strengths and weaknesses — and both have competitive advantages. Having less experience means having fewer constraints and less fear — after all, there is less to lose. But it also comes with a lack of wisdom and practical knowledge, as well as a tendency to make more mistakes along the way.\\\\nThe automotive industry has existed for over 100 years. A lot of the large OEMs have spent decades really honing how they can make their supply chain lean; build and sell cars on a large scale, at a high volume and cost-effectively; get the cars out to consumers 

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 80659.69it/s]


...

Hybrid Search Result: data: ['["id: 451726673013646285, distance: 0.032522473484277725, entity: {\'related_topics\': \'Advanced Manufacturing, Automotive\', \'pdf_links\': \'nan\', \'source_link\': \'https://www.ey.com/en_in/advanced-manufacturing/how-oems-can-achieve-the-best-of-both-worlds\', \'text\': \'In the next 10 years, you will undoubtedly see instances of new entrants being acquired by traditional original equipment manufacturers (OEMs). Both have problems that are not easy to solve. But when these two types of companies are paired together, these problems can be addressed.\\\\nBoth new entrants and traditional OEMs have strengths and weaknesses — and both have competitive advantages. Having less experience means having fewer constraints and less fear — after all, there is less to lose. But it also comes with a lack of wisdom and practical knowledge, as well as a tendency to make more mistakes along the way.\\\\nThe automotive industry has existed for over 100 years. A lot of

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 85948.85it/s]


Hybrid Search Result: data: ['["id: 451726673013646285, distance: 0.032522473484277725, entity: {\'source_link\': \'https://www.ey.com/en_in/advanced-manufacturing/how-oems-can-achieve-the-best-of-both-worlds\', \'text\': \'In the next 10 years, you will undoubtedly see instances of new entrants being acquired by traditional original equipment manufacturers (OEMs). Both have problems that are not easy to solve. But when these two types of companies are paired together, these problems can be addressed.\\\\nBoth new entrants and traditional OEMs have strengths and weaknesses — and both have competitive advantages. Having less experience means having fewer constraints and less fear — after all, there is less to lose. But it also comes with a lack of wisdom and practical knowledge, as well as a tendency to make more mistakes along the way.\\\\nThe automotive industry has existed for over 100 years. A lot of the large OEMs have spent decades really honing how they can make their supply chai

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 74898.29it/s]
INFO:httpx:HTTP Request: POST https://api.aimlapi.com/v1/chat/completions "HTTP/1.1 201 Created"


(('### Advanced Manufacturing\n\nAdvanced manufacturing refers to the use of innovative technologies and methodologies to improve products and processes in the manufacturing sector. It is a key component of the fourth industrial revolution, also known as Industry 4.0. This revolution is characterized by the integration of the physical, biological, and digital worlds, leveraging a range of emerging technological breakthroughs.\n\n#### Key Technologies in Advanced Manufacturing\n\n1. **Autonomous Capabilities**: These include self-operating machines and systems that can perform tasks without human intervention.\n2. **Artificial Intelligence (AI)**: AI is used to enhance decision-making processes, optimize operations, and predict maintenance needs.\n3. **Internet of Things (IoT)**: IoT connects devices and systems, enabling real-time data collection and analysis to improve efficiency and productivity.\n4. **Next-Generation Wireless Technologies**: These technologies, such as 5G, provide f

In [113]:
Response

(('### Advanced Manufacturing\n\nAdvanced manufacturing refers to the use of innovative technologies and methodologies to improve products and processes in the manufacturing sector. It is a key component of the fourth industrial revolution, also known as Industry 4.0. This revolution is characterized by the integration of the physical, biological, and digital worlds, leveraging a range of emerging technological breakthroughs.\n\n#### Key Technologies in Advanced Manufacturing\n\n1. **Autonomous Capabilities**: These include self-operating machines and systems that can perform tasks without human intervention.\n2. **Artificial Intelligence (AI)**: AI is used to enhance decision-making processes, optimize operations, and predict maintenance needs.\n3. **Internet of Things (IoT)**: IoT connects devices and systems, enabling real-time data collection and analysis to improve efficiency and productivity.\n4. **Next-Generation Wireless Technologies**: These technologies, such as 5G, provide f

In [114]:
display(Markdown(Response[0][0]))

### Advanced Manufacturing

Advanced manufacturing refers to the use of innovative technologies and methodologies to improve products and processes in the manufacturing sector. It is a key component of the fourth industrial revolution, also known as Industry 4.0. This revolution is characterized by the integration of the physical, biological, and digital worlds, leveraging a range of emerging technological breakthroughs.

#### Key Technologies in Advanced Manufacturing

1. **Autonomous Capabilities**: These include self-operating machines and systems that can perform tasks without human intervention.
2. **Artificial Intelligence (AI)**: AI is used to enhance decision-making processes, optimize operations, and predict maintenance needs.
3. **Internet of Things (IoT)**: IoT connects devices and systems, enabling real-time data collection and analysis to improve efficiency and productivity.
4. **Next-Generation Wireless Technologies**: These technologies, such as 5G, provide faster and more reliable communication, essential for real-time data transfer and remote operations.
5. **Nanotechnology**: This involves manipulating materials at the atomic or molecular level to create new materials and products with enhanced properties.
6. **Big Data**: The analysis of large datasets helps in making informed decisions, optimizing processes, and predicting trends.
7. **Blockchain**: Although still in its infancy, blockchain technology offers secure and transparent ways to track and verify transactions and data.
8. **Cloud Computing**: This provides scalable and flexible computing resources, enabling manufacturers to store and process large amounts of data efficiently.

#### Impact on the Automotive Industry

The automotive industry is significantly affected by these advancements. For instance, autonomous vehicles, AI-driven manufacturing processes, and IoT-enabled supply chains are transforming how cars are designed, produced, and delivered.

#### Challenges and Opportunities

- **Rapid Pace of Change**: The speed at which technology is evolving requires businesses to be agile and adaptable.
- **Customer Expectations**: With higher and ever-increasing customer expectations, manufacturers must continuously innovate to meet demand.
- **Volatile Demand**: Market demand can change rapidly, influenced by factors such as viral trends and social media influencers. This volatility necessitates flexible and responsive supply chains.

#### Conclusion

To thrive in this dynamic environment, businesses must embrace new capabilities that enable greater agility and responsiveness. Advanced manufacturing technologies offer the tools needed to navigate the complexities of modern supply chains and meet the challenges of Industry 4.0.

[Source: https://www.ey.com/en_in/advanced-manufacturing/why-the-fourth-time-s-the-charm-for-automotive]